In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('subset-2021-01-11-voter_fraud.csv')

In [3]:
data.columns

Index(['article', 'body', 'bodywithurls', 'comments', 'createdAt',
       'createdAtformatted', 'creator', 'datatype', 'depth', 'depthRaw',
       ...
       'urls.15.createdAt', 'urls.15.domain', 'urls.15.id', 'urls.15.long',
       'urls.15.metadata.length', 'urls.15.metadata.mimeType',
       'urls.15.metadata.site', 'urls.15.modified', 'urls.15.short',
       'urls.15.state'],
      dtype='object', length=420)

> * Subset the DataFrame to have `username`, `bodywithurls`, `followers`, and `following` columns

In [4]:
df=data[['username', 'bodywithurls', 'followers', 'following']]

In [15]:
df.shape

(91, 4)

> * Q. How many unique users are there in the dataset?

In [6]:
df['username'].nunique()

58

> * Let's print the first 5 rows of the DataFrame

In [13]:
df.head(5)

,username,bodywithurls,followers,following
0,Terryb158,Folks may be our last posts. Pogilosi just ask...,3500,5500
1,AlanBond7,Oh notre they want to investigate something. L...,100,243
2,Tifdog11,White trash pieces of shit who are going to be...,37,42
3,AlexaImmigrant,He has never lived in a totalitarian country; ...,0,9
4,Tifdog11,White trash pieces of shit who are going to be...,37,42


In [12]:
df.iloc[2]['bodywithurls'] == df.iloc[4]['bodywithurls']

True

> * Looks like there are duplicates in the dataset. Let's remove the duplicates.

In [16]:
df.drop_duplicates(keep='first', inplace=True)

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_1350/202288913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(keep='first', inplace=True)


In [17]:
df.shape

(65, 4)

> * Let's extract the mentions from the `bodywithurls` column and create a new column `mentions` with the mentions.

In [ ]:
pattern = re.compile(r'@')

In [18]:
df.head(5)

,username,bodywithurls,followers,following
0,Terryb158,Folks may be our last posts. Pogilosi just ask...,3500,5500
1,AlanBond7,Oh notre they want to investigate something. L...,100,243
2,Tifdog11,White trash pieces of shit who are going to be...,37,42
3,AlexaImmigrant,He has never lived in a totalitarian country; ...,0,9
5,Lovewell100,@sidneypowell @erictrump\n@laraleatrump @linwo...,48,148
